In [19]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [20]:
import shutil


In [21]:
image_dir = Path('.\image_data\database5\Fish_Dataset')


In [22]:
list(image_dir.glob(r'**/*.png'))

[WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/3-Figure1-1.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/383_2118.zt.800.800.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/383_5585.zt.360.240.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/4-Timing-of-the-upstream-migration-of-various-anadromous-arctic-char-populations-in.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/5b035a9e-16f4-4ee5-b980-9ee9d8738800_m.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/7-Figure3-1.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/A-Arctic-char-Salvelinus-alpinus-from-Happy-Creek-Lake-Aleknagik-AK-note-the-silver.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/A-small-Arctic-char-found-in-Ireland-top-a-pollan-found-in-Irish-freshwaters.png'),
 WindowsPath('image_data/database5/Fish_Dataset/ArcticCharFish/Alpine_Charr_preview_421a8eae

In [23]:
filepaths = list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

samples = []

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    if len(category_slice) < 350:
        samples.append(category_slice.sample(350, replace=True, random_state=1))
    else:
        samples.append(category_slice.sample(350, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)




In [24]:
image_df

,Filepath,Label
0,image_data\database5\Fish_Dataset\Red Sea Brea...,Red Sea Bream
1,image_data\database5\Fish_Dataset\Red-snapper\...,Red-snapper
2,image_data\database5\Fish_Dataset\Gilt-Head Br...,Gilt-Head Bream
3,image_data\database5\Fish_Dataset\Black Sea Sp...,Black Sea Sprat
4,image_data\database5\Fish_Dataset\Trout\Trout\...,Trout
...,...,...
6645,image_data\database5\Fish_Dataset\Black Sea Sp...,Black Sea Sprat
6646,image_data\database5\Fish_Dataset\Striped Red ...,Striped Red Mullet
6647,image_data\database5\Fish_Dataset\SardineFish\...,SardineFish
6648,image_data\database5\Fish_Dataset\ArcticCharFi...,ArcticCharFish


In [25]:
labels

0        ArcticCharFish
1        ArcticCharFish
2        ArcticCharFish
3        ArcticCharFish
4        ArcticCharFish
              ...      
27089              tuna
27090              tuna
27091              tuna
27092              tuna
27093              tuna
Name: Label, Length: 27094, dtype: object

In [26]:
image_df


,Filepath,Label
0,image_data\database5\Fish_Dataset\Red Sea Brea...,Red Sea Bream
1,image_data\database5\Fish_Dataset\Red-snapper\...,Red-snapper
2,image_data\database5\Fish_Dataset\Gilt-Head Br...,Gilt-Head Bream
3,image_data\database5\Fish_Dataset\Black Sea Sp...,Black Sea Sprat
4,image_data\database5\Fish_Dataset\Trout\Trout\...,Trout
...,...,...
6645,image_data\database5\Fish_Dataset\Black Sea Sp...,Black Sea Sprat
6646,image_data\database5\Fish_Dataset\Striped Red ...,Striped Red Mullet
6647,image_data\database5\Fish_Dataset\SardineFish\...,SardineFish
6648,image_data\database5\Fish_Dataset\ArcticCharFi...,ArcticCharFish


In [27]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)


In [28]:
def create_category_folders(base_dir, categories):
    for category in categories:
        category_path = os.path.join(base_dir, category)
        os.makedirs(category_path, exist_ok=True)

def move_files(df, base_dir):
    for _, row in df.iterrows():
        category = row['Label']
        file_path = row['Filepath']
        destination_dir = os.path.join(base_dir, category)
        shutil.copy(file_path, destination_dir)

def organize_train_test_data(train_df, test_df, base_dir='organized_data'):
    # Create base directories for training and testing
    train_dir = os.path.join(base_dir, 'train')
    test_dir = os.path.join(base_dir, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get unique categories from the DataFrame
    categories = train_df['Label'].unique()

    # Create category folders for both train and test directories
    create_category_folders(train_dir, categories)
    create_category_folders(test_dir, categories)

    # Move the training and testing files to the respective directories
    move_files(train_df, train_dir)
    move_files(test_df, test_dir)

    print("Files have been organized into train and test folders.")

# Step 7: Organize the train and test data
organize_train_test_data(train_df, test_df, base_dir='organized_data')


KeyboardInterrupt: 

In [32]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)


In [33]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)


Found 3724 validated image filenames belonging to 19 classes.
Found 931 validated image filenames belonging to 19 classes.
Found 1995 validated image filenames belonging to 19 classes.


In [34]:
train_class_indices = train_images.class_indices
print("Class indices in training set:", train_class_indices)

# Reverse the dictionary to map indices to class names
index_to_class = {v: k for k, v in train_class_indices.items()}
print("Index to class mapping:", index_to_class)

Class indices in training set: {'ArcticCharFish': 0, 'Black Sea Sprat': 1, 'Gilt-Head Bream': 2, 'Hourse Mackerel': 3, 'Red Mullet': 4, 'Red Sea Bream': 5, 'Red-snapper': 6, 'SardineFish': 7, 'Sea Bass': 8, 'Shrimp': 9, 'Striped Red Mullet': 10, 'TilapiaFish': 11, 'Trout': 12, 'bassfish': 13, 'catfish': 14, 'haddockfish': 15, 'salmon': 16, 'swordfish': 17, 'tuna': 18}
Index to class mapping: {0: 'ArcticCharFish', 1: 'Black Sea Sprat', 2: 'Gilt-Head Bream', 3: 'Hourse Mackerel', 4: 'Red Mullet', 5: 'Red Sea Bream', 6: 'Red-snapper', 7: 'SardineFish', 8: 'Sea Bass', 9: 'Shrimp', 10: 'Striped Red Mullet', 11: 'TilapiaFish', 12: 'Trout', 13: 'bassfish', 14: 'catfish', 15: 'haddockfish', 16: 'salmon', 17: 'swordfish', 18: 'tuna'}


In [35]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

base_model.trainable = False


In [36]:
inputs = base_model.input

x = tf.keras.layers.Dense(128, activation='relu')(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(19, activation='softmax')(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)




In [37]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100


d:\Projuct_\Project_code\python_for_modeling\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 58/117 ━━━━━━━━━━━━━━━━━━━━ 32s 547ms/step - accuracy: 0.3665 - loss: 2.1011

d:\Projuct_\Project_code\python_for_modeling\.venv\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


117/117 ━━━━━━━━━━━━━━━━━━━━ 94s 730ms/step - accuracy: 0.4885 - loss: 1.6964 - val_accuracy: 0.7669 - val_loss: 0.7229
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 718ms/step - accuracy: 0.8279 - loss: 0.5234 - val_accuracy: 0.7981 - val_loss: 0.6350
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 718ms/step - accuracy: 0.8955 - loss: 0.3420 - val_accuracy: 0.8034 - val_loss: 0.6300
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 84s 706ms/step - accuracy: 0.9329 - loss: 0.2416 - val_accuracy: 0.8077 - val_loss: 0.6667
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 85s 717ms/step - accuracy: 0.9685 - loss: 0.1430 - val_accuracy: 0.8002 - val_loss: 0.7086
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 85s 714ms/step - accuracy: 0.9757 - loss: 0.1065 - val_accuracy: 0.8142 - val_loss: 0.7084


In [38]:
# Unfreeze the top layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Freeze all but the last 20 layers
    layer.trainable = False

# Re-compile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with fine-tuning
history_fine_tune = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,  # Fine-tuning for fewer epochs
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)


Epoch 1/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 111s 825ms/step - accuracy: 0.7272 - loss: 0.8658 - val_accuracy: 0.8110 - val_loss: 0.6116
Epoch 2/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 95s 799ms/step - accuracy: 0.8958 - loss: 0.4004 - val_accuracy: 0.8110 - val_loss: 0.6073
Epoch 3/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 98s 823ms/step - accuracy: 0.9259 - loss: 0.3128 - val_accuracy: 0.8120 - val_loss: 0.6036
Epoch 4/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 96s 806ms/step - accuracy: 0.9385 - loss: 0.2747 - val_accuracy: 0.8120 - val_loss: 0.5988
Epoch 5/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 95s 802ms/step - accuracy: 0.9533 - loss: 0.2274 - val_accuracy: 0.8131 - val_loss: 0.5934
Epoch 6/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 95s 800ms/step - accuracy: 0.9572 - loss: 0.2203 - val_accuracy: 0.8163 - val_loss: 0.5903
Epoch 7/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 96s 804ms/step - accuracy: 0.9563 - loss: 0.2020 - val_accuracy: 0.8153 - val_loss: 0.5834
Epoch 8/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 98s 826ms/step - accuracy: 0.9719 - loss: 

In [39]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

    Test Loss: 0.57573
Test Accuracy: 83.06%


In [40]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 4,911,739 (18.74 MB)

 Trainable params: 1,235,411 (4.71 MB)

 Non-trainable params: 1,205,504 (4.60 MB)

 Optimizer params: 2,470,824 (9.43 MB)

In [67]:
import matplotlib.pyplot as plt


In [47]:
kearas_file = "model/Fish_use.h5"
tf.keras.models.save_model(model,kearas_file)

In [48]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
with open("./model/Fish_use.tflite","wb") as f:
    f.write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\KK\AppData\Local\Temp\tmppcy7yddu\assets


INFO:tensorflow:Assets written to: C:\Users\KK\AppData\Local\Temp\tmppcy7yddu\assets


Saved artifact at 'C:\Users\KK\AppData\Local\Temp\tmppcy7yddu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_313')
Output Type:
  TensorSpec(shape=(None, 19), dtype=tf.float32, name=None)
Captures:
  1757900101328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900101520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900102672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900101904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900101712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900102480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900102288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900104592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900103440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1757900103056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  175